In [255]:
import requests as req 
from bs4 import BeautifulSoup   
import time
import json
import tqdm
import numpy as np
import pandas as pd

websites = {'TASS' : ['https://tass.ru/rss/v2.xml', ''],
            'LENTA.RU' : ['https://lenta.ru/rss', ''],
            'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']
           }

data = {"data" : []  }
all_data = {"data" : []  }
df = pd.DataFrame(data['data'])
df_all = pd.DataFrame(all_data['data'])


def parser(df, df_all):        
        
    for site in range(len(websites)):
        
        exitFlag=False
        
        data = {"data" : [] }
        all_data = {"data" : [] }
       
        url = list(websites.values())[site][0]
                
        resp = req.get(url)
        soup  = BeautifulSoup(resp.content, features="lxml")

        tag = soup.find_all('item')
        
        for item in tag:
            
            title = item.find('title')
            
            if title.text != list(websites.values())[site][1]: 
                exitFlag=True 
                
                category = item.find('category')

                date = item.find('pubdate')

                data['data'].append({"title" : title.text, "category": category.text,
                                     "pubdate" : date.text, "site" : list(websites.keys())[site]
                                    })
                all_data['data'].append({"site" : list(websites.keys())[site], "item" : item.text})


                with open(f"all_data_{pd.Timestamp('today'):%d-%m-%Y-%H-%M}.json", "w", encoding="utf-8") as file:
                      json.dump(all_data, file, ensure_ascii = False)
                        
            else:
                break                                
                
        if(exitFlag):
            df = pd.concat([pd.DataFrame(data['data']), df], ignore_index=True)
            df_all = pd.concat([pd.DataFrame(all_data['data']), df_all], ignore_index=True)        
            websites[list(websites.keys())[site]][1] = df.iloc[0]['title']   
          
    return df, df_all

df, df_all  = parser(df, df_all) 
df.sample(10)

/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup cons

,title,category,pubdate,site
24,Медведев допустил уничтожение миротворцев НАТО...,Политика,"Fri, 31 Mar 2023 16:54:25 +0300",ВЕДОМОСТИ
444,"В Бахчисарае откроется выставка, посвященная м...",В стране,"Sat, 01 Apr 2023 10:08:37 +0300",TASS
54,Лукашенко призвал прекратить боевые действия н...,Политика,"Fri, 31 Mar 2023 11:52:24 +0300",ВЕДОМОСТИ
302,Рогов назвал возможные сроки штурма ВСУ Киево-...,Бывший СССР,"Fri, 31 Mar 2023 23:03:00 +0300",LENTA.RU
163,Медведев призвал нарастить производство боепри...,Политика,"Wed, 29 Mar 2023 18:59:34 +0300",ВЕДОМОСТИ
145,ВТБ предупредил о схеме мошенничества с «блоки...,Финансы,"Thu, 30 Mar 2023 10:27:26 +0300",ВЕДОМОСТИ
484,<![CDATA[На Чукотке ожидается усиление ветра д...,Происшествия,"Sat, 01 Apr 2023 05:36:10 +0300",TASS
30,Адвокат Трампа назвал сделку со следствием мал...,Политика,"Fri, 31 Mar 2023 15:47:19 +0300",ВЕДОМОСТИ
98,При столкновении военных вертолетов в США поги...,Общество,"Thu, 30 Mar 2023 18:10:18 +0300",ВЕДОМОСТИ
408,На Кубани в ДТП с грузовиком погибли три человека,Происшествия,"Sat, 01 Apr 2023 11:45:16 +0300",TASS


In [256]:
if len(df['title'].unique()) < len(df):
    print('There are duplicates in the data')
    df = df.drop_duplicates('title', keep='first')

There are duplicates in the data


In [257]:
if df['title'].isnull().any():
    print('There are missing values in this dataset')
    df = df[df['title'].isnull()==False]

In [258]:
df.head()

,title,category,pubdate,site
0,Сбербанк повысил комиссии для магазинов за QR-...,Финансы,"Sat, 01 Apr 2023 11:36:32 +0300",ВЕДОМОСТИ
1,Шойгу заявил о кратном увеличении выпуска сред...,Политика,"Sat, 01 Apr 2023 11:01:33 +0300",ВЕДОМОСТИ
2,"МВФ утвердил выделение кредита Украине на $15,...",Финансы,"Sat, 01 Apr 2023 10:32:37 +0300",ВЕДОМОСТИ
3,Наместник Киево-Печерской лавры сообщил о пред...,Политика,"Sat, 01 Apr 2023 09:58:44 +0300",ВЕДОМОСТИ
4,Кабмин увеличил уставный капитал РЖД на 10 млр...,Экономика,"Sat, 01 Apr 2023 09:28:22 +0300",ВЕДОМОСТИ


3) РАБОТАЕМ С КАТЕГОРИЯМИ

## Общее количество новостей из всех источников по данной категории за все время

In [263]:
df_1 = pd.DataFrame(df['category'].value_counts())
df_1

,category
Общество,79
Политика,78
Мир,45
Россия,33
Экономика,32
Бывший СССР,27
Спорт,24
Бизнес,21
Финансы,20
Происшествия,20


## Количество новостей данной категории для каждого из источников за все время

In [264]:
new_df  = df.iloc[:, [0,1,3]]
df_category = pd.DataFrame(new_df.groupby('site')['category'].value_counts())
df_category.rename(columns={'category': 'Count'}, inplace=True)

df_category

Count
site      category                     
LENTA.RU  Мир                        45
          Россия                     33
          Бывший СССР                27
          Экономика                  15
          Спорт                      13
          Забота о себе              12
          Среда обитания             10
          Силовые структуры           9
          Из жизни                    7
          Интернет и СМИ              7
          Наука и техника             7
          Ценности                    5
          Культура                    3
          Моя страна                  3
          Путешествия                 3
TASS      Общество                   23
          Происшествия               20
          Экономика и бизнес         13
          Международная панорама     11
          Спорт                      11
          В стране                    6
          Армия и ОПК                 4
          Москва                      4
          Политика                    4
          Культура                    3
          Северо-Запад                1
ВЕДОМОСТИ Политика                   74
          Общество                   56
          Бизнес                     21
          Финансы                    20
          Экономика                  17
          Технологии                  5
          Авто                        3
          Медиа                       3
          Недвижимость                1

## Общее количество новостей из всех источников по данной категории за последние сутки

In [235]:
from datetime import datetime as dt
df_news = df.copy()

In [236]:
df_news['date_of_week'] = 1
df_news['date_and_time'] = 1

for index, row in enumerate(df_news['pubdate'].str.split(',')):
    #print(row)
    df_news['date_of_week'][index] = row[0]
    df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])

/var/folders/vh/_yg98lps1dvdkkx_jvfjzsm80000gp/T/ipykernel_1813/3737547308.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date_of_week'][index] = row[0]
/var/folders/vh/_yg98lps1dvdkkx_jvfjzsm80000gp/T/ipykernel_1813/3737547308.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])


In [237]:
for index, row in enumerate(df_news['pubdate'].str.split(',')):
    #print(row)
    df_news['date_of_week'][index] = row[0]
    df_news['date_and_time'][index] = " ".join(row[1].split()[:-1])


In [216]:
df_news.sample(7)

,title,category,pubdate,site,date_of_week,date_and_time
260,В краснодарском парке зацвела сакура,Моя страна,"Fri, 31 Mar 2023 20:10:00 +0300",LENTA.RU,Fri,31 Mar 2023 20:10:00
120,Песков рассказал о возможности эвакуации монах...,Политика,"Thu, 30 Mar 2023 13:44:34 +0300",ВЕДОМОСТИ,Thu,30 Mar 2023 13:44:34
206,Скидки на российскую нефть начали уменьшать,Экономика,"Sat, 01 Apr 2023 00:07:15 +0300",LENTA.RU,Sat,01 Apr 2023 00:07:15
411,Солисты оперы Мариинки дадут концерт в честь ю...,Культура,"Sat, 01 Apr 2023 00:10:27 +0300",TASS,Sat,01 Apr 2023 00:02:40
382,Пропавшего 15-летнего российского школьника на...,Россия,"Fri, 31 Mar 2023 15:57:00 +0300",LENTA.RU,Fri,31 Mar 2023 15:52:00
158,Медведев призвал нарастить производство боепри...,Политика,"Wed, 29 Mar 2023 18:59:34 +0300",ВЕДОМОСТИ,Wed,29 Mar 2023 18:59:34
487,"В ООН заявили, что генсек не вмешивается в спо...",Международная панорама,"Fri, 31 Mar 2023 20:50:06 +0300",TASS,Fri,31 Mar 2023 20:47:17


In [238]:
df_news['date_and_time'] = dt.now()
df_news = df_news.drop(columns = ['pubdate'])

df_news

,title,category,site,date_of_week,date_and_time
0,Орбан оценил угрозу начала третьей мировой войны,Политика,ВЕДОМОСТИ,Fri,2023-04-01 12:06:15.008860
1,Мишустин подписал распоряжение о создании ремо...,Политика,ВЕДОМОСТИ,Fri,2023-04-01 12:06:15.008860
2,Ростовский губернатор сообщил об устранении по...,Общество,ВЕДОМОСТИ,Fri,2023-04-01 12:06:15.008860
3,Минюст включил в реестр иноагентов музыканта М...,Общество,ВЕДОМОСТИ,Fri,2023-04-01 12:06:15.008860
4,Руководство МФТИ отказалось от обязательного и...,Общество,ВЕДОМОСТИ,Fri,2023-04-01 12:06:15.008860
...,...,...,...,...,...
495,"Минюст США подал иск против компании, чей поез...",Общество,TASS,Fri,2023-04-01 12:06:15.008860
496,"""Ростелеком"" получил косвенный контроль в пров...",Экономика и бизнес,TASS,Fri,2023-04-01 12:06:15.008860
497,Постпред Белоруссии при ООН предложил Западу р...,Международная панорама,TASS,Fri,2023-04-01 12:06:15.008860
498,"Эксперт считает, что вступление Финляндии в НА...",Политика,TASS,1,2023-04-01 12:06:15.008860


## Среднее количество публикаций по данной категории в сутки

In [239]:
df_average = df_news.copy()

In [251]:
df_average['date_and_time'] = dt.now()

In [253]:
df_average['category']. value_counts ()['Политика']/2

41.5

## День, в который было сделано максимальное кол-во публикаций по данной категории

## Количество публикаций новостей данной категории по дням недели

In [273]:
conda install psycopg2

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [287]:
import psycopg2

conn = psycopg2.connect(
    database="final project",
    user="postgres",
    password="localhost",
    host='127.0.0.1', 
    port='5432'
)



In [288]:
print("Database opened successfully")
cur = conn.cursor()  
cur.execute('''CREATE TABLE New  
     (CATEGORY CHAR PRIMARY KEY NOT NULL,
     TITLE TEXT NOT NULL,
     DATE_OF_WEEK CHAR(50),
     DATE_AND_TIME TIMESTAMP NOT NULL,
     SITE CHAR(50));''')

print("Table created successfully")
conn.commit()  
conn.close()


Database opened successfully
Table created successfully
